## Introduction

This notebook uses historical futures data of 25 different commodities and analyzes portfolio allocations based on two factors present in the commodity market:

1. momentum
2. basis/term-structure
3. idiosyncratic volatility
4. skewness

I analyze portfolio sorts based on each single signal and combined signals. I also compare properties of the cross-sectional and time-series signaling.

**Sample Description**
* 25 commodities
* Sample spans from 1980 to 2022
* Price data is retrieved from Thomson Reuters Datastream and the Chicago Mercantile Exchange (CME)

**References**
* Miffre, J., & Rallis, G. (2007). Momentum strategies in commodity futures markets. Journal of Banking & Finance, 31(6), 1863-1886.
https://risk.edhec.edu/sites/risk/files/pdf/RISKArticle.2007-11-16.4731/attachments/Momentum%20Strategies%20Commodity%20Futures.pdf
* Fuertes, A. M., Miffre, J., & Fernandez‐Perez, A. (2015). Commodity strategies based on momentum, term structure, and idiosyncratic volatility. Journal of Futures Markets, 35(3), 274-297.
https://risk.edhec.edu/sites/risk/files/edhec-working-paper-commodity-strategies-based-on-momentum_1368521019664_0.pdf
* Fuertes, A. M., Miffre, J., & Rallis, G. (2010). Tactical allocation in commodity futures markets: Combining momentum and term structure signals. Journal of Banking & Finance, 34(10), 2530-2548.
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.458.5618&rep=rep1&type=pdf
* Miffre, J. (2016). Long-short commodity investing: A review of the literature. Journal of Commodity Markets, 1(1), 3-13.
https://risk.edhec.edu/sites/risk/files/edhec_working_paper_long-short_commodity_investing.pdf
* Fernandez-Perez, A., Frijns, B., Fuertes, A. M., & Miffre, J. (2018). The skewness of commodity futures returns. Journal of Banking & Finance, 86, 143-158.
https://openaccess.city.ac.uk/id/eprint/17843/1/SSRN-id2671165_July2017.pdf
* Gorton, G. B., Hayashi, F., & Rouwenhorst, K. G. (2013). The fundamentals of commodity futures returns. Review of Finance, 17(1), 35-105.
https://www.nber.org/system/files/working_papers/w13249/w13249.pdf